# Analiza lige NBA
## Predstavitev podatkov
V spodnjem dokumentu bom s pomočjo tabel in grafov predstavil podatke iz košarkaške lige NBA za lansko sezono. Na podlagi njih bom pogledal ali lahko lahko z dovolj veliko gotovostjo delovne hipoteze zavrnem.
Podatke sem uvozil s pomočjo datotek `obdelava_podatkov.py` in `zajem_podatkov.py`. S pomočjo teh skript iz spletne strani [Basketball reference](https://www.basketball-reference.com/) poberem HTML datoteke, ki jih s pomočjo regularnih izrazov obdelam in zapišem v datoteki `players.csv` in `teams.csv`. Obe tabeli prikazujeta podatke statistik, ki jih posamezna ekipa ali pa igralec dosežejo na tekmo. 
Poleg tega sem še ročno uvozil še `teams_advanced.csv`, ki jo bom uporabil samo za informacije o zmagah in porazih v lanski sezoni. 

Datoteke prvo uvozimo s pomočjo pandas knjižnice.

In [400]:
import pandas as pd 

players = pd.read_csv("players.csv", index_col="id")
teams = pd.read_csv("teams.csv", index_col="team_id")
teams_advanced = pd.read_csv("teams_advanced.csv")

Preverimo ali podatki izgledajo vredu.

In [401]:
players.head()

,player,position,team_id,fg,fga,trb,ast,stl,blk,tov,pts
id,,,,,,,,,,,
1,Precious Achiuwa,PF,MIA,2.0,3.7,3.4,0.5,0.3,0.5,0.7,5.0
2,Jaylen Adams,PG,MIL,0.1,1.1,0.4,0.3,0.0,0.0,0.0,0.3
3,Steven Adams,C,NOP,3.3,5.3,8.9,1.9,0.9,0.7,1.3,7.6
4,Bam Adebayo,C,MIA,7.1,12.5,9.0,5.4,1.2,1.0,2.6,18.7
5,LaMarcus Aldridge,C,SAS,5.5,11.8,4.5,1.7,0.4,0.9,1.0,13.7


In [402]:
teams.head()

,team,games,minutes_played,fg,fga,fg3,fga3,ft,fta,orb,drb,ast,stl,blk,tov,pf,pts
team_id,,,,,,,,,,,,,,,,,
MIL,Milwaukee Bucks,72.0,240.7,44.7,91.8,14.4,37.1,16.2,21.4,10.3,37.8,25.5,8.1,4.6,13.8,17.3,120.1
BRK,Brooklyn Nets,72.0,241.7,43.1,87.3,14.2,36.1,18.1,22.5,8.9,35.5,26.8,6.7,5.3,13.5,19.0,118.6
WAS,Washington Wizards,72.0,241.7,43.2,90.9,10.2,29.0,20.1,26.2,9.7,35.5,25.5,7.3,4.1,14.4,21.6,116.6
UTA,Utah Jazz,72.0,241.0,41.3,88.1,16.7,43.0,17.2,21.5,10.6,37.6,23.7,6.6,5.2,14.2,18.5,116.4
POR,Portland Trail Blazers,72.0,240.3,41.3,91.1,15.7,40.8,17.8,21.6,10.6,33.9,21.3,6.9,5.0,11.1,18.9,116.1


In [403]:
teams_advanced.head()

,Rk,Team,Age,W,L,PW,PL,MOV,SOS,SRS,...,FT/FGA,Unnamed: 22,eFG%.1,TOV%.1,DRB%,FT/FGA.1,Unnamed: 27,Arena,Attend.,Attend./G
0,1,Utah Jazz,28.5,52,20,55,17,9.25,-0.29,8.97,...,0.195,NaN,0.507,10.3,79.3,0.159,NaN,Vivint Smart Home Arena,151300.0,4203.0
1,2,Los Angeles Clippers,28.8,47,25,49,23,6.18,-0.16,6.02,...,0.186,NaN,0.531,11.9,79.1,0.186,NaN,STAPLES Center,13901.0,386.0
2,3,Phoenix Suns,26.6,51,21,49,23,5.82,-0.15,5.67,...,0.177,NaN,0.534,12.4,78.5,0.194,NaN,Phoenix Suns Arena,104027.0,2890.0
3,4,Milwaukee Bucks,28.1,46,26,48,24,5.89,-0.32,5.57,...,0.177,NaN,0.536,11.5,79.7,0.157,NaN,Fiserv Forum,64780.0,1799.0
4,5,Philadelphia 76ers,27.1,49,23,48,24,5.58,-0.31,5.28,...,0.225,NaN,0.521,13.8,78.2,0.200,NaN,Wells Fargo Center,68583.0,1905.0


Ker potrebujem samo stolpce z imenom ekipe, število zmag in porazov, ostale stolpce zbrišem. 

In [404]:
teams_advanced = teams_advanced[['Team', 'W', 'L']]
teams_advanced.columns = ['team', 'w', 'l']
teams_advanced.head()


,team,w,l
0,Utah Jazz,52,20
1,Los Angeles Clippers,47,25
2,Phoenix Suns,51,21
3,Milwaukee Bucks,46,26
4,Philadelphia 76ers,49,23


Zdaj bom še odstranil ime ekipe, saj je dovolj da naredim novo tabelo z id-ji in imeni ekip.

In [405]:
team_ids = teams['team'].copy()
teams = teams.drop("team", axis=1)

# Vzorčna statistika
V prvem delu analize si bom pogledal kakšen vpliv imajo osnovne statistike pri uspešnosti ekip. Spodnja tabela prikazuje povprečje, mediano in standardni odklon glavnih statistik tekme glede na vse ekipe.

In [406]:
mean_values = teams.mean()
median_values = teams.median()
std_values = teams.std()
sample_statistic = pd.concat([mean_values, median_values, std_values], axis=1)
sample_statistic = sample_statistic[1:]
sample_statistic.rename(columns={0:'mean', 1:'median', 2:'standard_deviation'})


,mean,median,standard_deviation
minutes_played,241.420000,241.55,0.859992
fg,41.220000,41.30,1.702412
fga,88.416667,88.40,1.970486
fg3,12.706667,12.75,1.694399
fga3,34.636667,34.40,3.782125
ft,16.970000,16.60,1.348856
fta,21.816667,21.40,1.956260
orb,9.826667,9.70,0.856188
drb,34.470000,34.60,1.396585
ast,24.806667,24.55,1.835650


Spodnja tabela prikazuje maksimalne vrednosti glede na tekme, ter katera ekipa jih ima. 

In [407]:
max_values = teams.max()
max_id = teams.idxmax()
max_teams = pd.concat([max_id, max_values], axis=1)
max_teams.rename(columns={0: 'max_id', 1: 'max_value'})

,max_id,max_value
games,MIL,72.0
minutes_played,PHO,242.8
fg,MIL,44.7
fga,MIL,91.8
fg3,UTA,16.7
fga3,UTA,43.0
ft,WAS,20.1
fta,WAS,26.2
orb,NOP,11.7
drb,MIL,37.8


In [408]:
# TODO poglej katere ekipe so nad povprečjem, oziroma nad mediano, mogoče oboje. Poglej ali je potrebno dodati tudi csv vseh tekem in narediti regex iz tega

In [409]:
# TODO poglej igralce, ki najbolj izstopajo in ali imajo zaradi tega vpliv na ekipe, oziroma ali obstaja kakšna kategorija ki je korelirana z zmagami

In [410]:
# TODO nariši še par grafov za korelacije

In [411]:
# TODO machine learning za napoved zmag. Premisli kaj je najbolj smiselna statistika za vpisovat in kaj napoveduje. Pametno bi bilo da bi bila numerična, ki vrača numerično... Kategorično je problem